#손톱모양판별

In [ ]:
import cv2
import numpy as np

# 손톱 shape 확률 매핑 테이블
shape_table = {
    "long-narrow": {"square":0.606952, "round":0.037433, "oval":0.068627, "squoval":0.149733, "almond":0.137255},
    "long-wide":   {"square":0.314815, "round":0.064815, "oval":0.25, "squoval":0.0, "almond":0.37037},
    "mid-narrow":  {"square":0.50947,  "round":0.039773, "oval":0.145833, "squoval":0.159091, "almond":0.145833},
    "mid-wide":    {"square":0.236842, "round":0.061404, "oval":0.350877, "squoval":0.0, "almond":0.350877},
    "short-narrow":{"square":0.334775, "round":0.361643, "oval":0.07134, "squoval":0.208462, "almond":0.02378},
    "short-wide":  {"square":0.150052, "round":0.389927, "oval":0.259605, "squoval":0.021807, "almond":0.178609}
}

# -----------------------------------------------
# 1) 이미지 로드 + 이진화
# -----------------------------------------------
img_path = "/content/test.jpg"   # ← Soyeon 이미지
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
_, th = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

# -----------------------------------------------
# 2) 컨투어(손톱 5개) 탐지
# -----------------------------------------------
contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 손가락 순서 정렬 (x 좌표 기준)
contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])

# -----------------------------------------------
# 3) width 값으로 narrow/wide 기준을 만들기 위해 전체 width 수집
# -----------------------------------------------
widths = [cv2.boundingRect(c)[2] for c in contours]
width_median = np.median(widths)

# -----------------------------------------------
# 4) 각 손톱 shape 계산
# -----------------------------------------------
results = []

for idx, cnt in enumerate(contours):
    x, y, w, h = cv2.boundingRect(cnt)
    aspect = h / w

    # 길이 분류
    if aspect < 1.2:
        length = "short"
    elif aspect < 1.6:
        length = "mid"
    else:
        length = "long"

    # 폭 분류
    width_type = "narrow" if w < width_median else "wide"

    key = f"{length}-{width_type}"

    # shape 후보 중 확률 가장 높은 것 선택
    probs = shape_table[key]
    best_shape = max(probs, key=probs.get)

    results.append({
        "Finger": idx+1,
        "Width": w,
        "Height": h,
        "Aspect": round(aspect,3),
        "LengthType": length,
        "WidthType": width_type,
        "Category": key,
        "Shape": best_shape
    })

# 결과 출력
for r in results:
    print(r)


{'Finger': 1, 'Width': 3024, 'Height': 3024, 'Aspect': 1.0, 'LengthType': 'short', 'WidthType': 'wide', 'Category': 'short-wide', 'Shape': 'round'}
{'Finger': 2, 'Width': 1, 'Height': 1, 'Aspect': 1.0, 'LengthType': 'short', 'WidthType': 'wide', 'Category': 'short-wide', 'Shape': 'round'}
{'Finger': 3, 'Width': 2, 'Height': 1, 'Aspect': 0.5, 'LengthType': 'short', 'WidthType': 'wide', 'Category': 'short-wide', 'Shape': 'round'}
{'Finger': 4, 'Width': 2, 'Height': 2, 'Aspect': 1.0, 'LengthType': 'short', 'WidthType': 'wide', 'Category': 'short-wide', 'Shape': 'round'}
{'Finger': 5, 'Width': 1, 'Height': 1, 'Aspect': 1.0, 'LengthType': 'short', 'WidthType': 'wide', 'Category': 'short-wide', 'Shape': 'round'}
{'Finger': 6, 'Width': 3, 'Height': 2, 'Aspect': 0.667, 'LengthType': 'short', 'WidthType': 'wide', 'Category': 'short-wide', 'Shape': 'round'}
{'Finger': 7, 'Width': 1, 'Height': 2, 'Aspect': 2.0, 'LengthType': 'long', 'WidthType': 'wide', 'Category': 'long-wide', 'Shape': 'almond'}

#순위별로 출력

In [ ]:
# shape 후보 전체를 확률 순위대로
sorted_shapes = sorted(probs.items(), key=lambda x: x[1], reverse=True)

results.append({
    "Finger": idx+1,
    "Width": w,
    "Height": h,
    "Aspect": round(aspect,3),
    "LengthType": length,
    "WidthType": width_type,
    "Category": key,
    "TopShape": sorted_shapes[0][0],     # 1위 shape
    "AllShapesRanked": sorted_shapes      # 전체 shape 추천 순위
})


In [ ]:
import cv2
import numpy as np

shape_table = {
    "long-narrow": {"square":0.606952, "round":0.037433, "oval":0.068627, "squoval":0.149733, "almond":0.137255},
    "long-wide":   {"square":0.314815, "round":0.064815, "oval":0.25, "squoval":0.0, "almond":0.37037},
    "mid-narrow":  {"square":0.50947,  "round":0.039773, "oval":0.145833, "squoval":0.159091, "almond":0.145833},
    "mid-wide":    {"square":0.236842, "round":0.061404, "oval":0.350877, "squoval":0.0, "almond":0.350877},
    "short-narrow":{"square":0.334775, "round":0.361643, "oval":0.07134, "squoval":0.208462, "almond":0.02378},
    "short-wide":  {"square":0.150052, "round":0.389927, "oval":0.259605, "squoval":0.021807, "almond":0.178609}
}

img_path = "/content/test.jpg"
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
_, th = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])

widths = [cv2.boundingRect(c)[2] for c in contours]
width_median = np.median(widths)

results = []

for idx, cnt in enumerate(contours):
    x, y, w, h = cv2.boundingRect(cnt)
    aspect = h / w

    # 길이 분류
    if aspect < 1.2:
        length = "short"
    elif aspect < 1.6:
        length = "mid"
    else:
        length = "long"

    # 폭 분류
    width_type = "narrow" if w < width_median else "wide"

    key = f"{length}-{width_type}"

    probs = shape_table[key]

    # 전체 shape 추천 순위
    sorted_shapes = sorted(probs.items(), key=lambda x: x[1], reverse=True)

    results.append({
        "Finger": idx+1,
        "Width": w,
        "Height": h,
        "Aspect": round(aspect,3),
        "LengthType": length,
        "WidthType": width_type,
        "Category": key,
        "TopShape": sorted_shapes[0][0],
        "AllShapesRanked": sorted_shapes
    })

# 결과 출력
for r in results:
    print(f"\n### Finger {r['Finger']} ({r['Category']})")
    print("추천 Shape 순위:")
    for shape, prob in r["AllShapesRanked"]:
        print(f" - {shape}: {prob:.4f}")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.

### Finger 8552 (short-wide)
추천 Shape 순위:
 - round: 0.3899
 - oval: 0.2596
 - almond: 0.1786
 - square: 0.1501
 - squoval: 0.0218

### Finger 8553 (short-wide)
추천 Shape 순위:
 - round: 0.3899
 - oval: 0.2596
 - almond: 0.1786
 - square: 0.1501
 - squoval: 0.0218

### Finger 8554 (short-wide)
추천 Shape 순위:
 - round: 0.3899
 - oval: 0.2596
 - almond: 0.1786
 - square: 0.1501
 - squoval: 0.0218

### Finger 8555 (short-wide)
추천 Shape 순위:
 - round: 0.3899
 - oval: 0.2596
 - almond: 0.1786
 - square: 0.1501
 - squoval: 0.0218

### Finger 8556 (short-wide)
추천 Shape 순위:
 - round: 0.3899
 - oval: 0.2596
 - almond: 0.1786
 - square: 0.1501
 - squoval: 0.0218

### Finger 8557 (short-wide)
추천 Shape 순위:
 - round: 0.3899
 - oval: 0.2596
 - almond: 0.1786
 - square: 0.1501
 - squoval: 0.0218

### Finger 8558 (short-wide)
추천 Shape 순위:
 - round: 0.3899
 - oval: 0.2596
 - almond: 0.1786
 - square: 0.1501
 - squoval: 0.0218

### Finger 8559 (short-wide)
추천 Shape 순위:
 - r